### Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

In [209]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
import json
from datetime import datetime as dt
import time

#### Задание 1.
Обязательная часть\
Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).\
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:\
\
KEYWORDS = ['python', 'парсинг']\
\
Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).\
В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

In [217]:
"""
Сделал две функции:
1) Проверка на наличие необходимого вхождения слова (проверял в анонсе, заголовках и тегах https://prnt.sc/13dytjc)
2) И собственно получение информации с доступной preview-информации
"""

KEYWORDS = ['python', 'парсинг']

def check_keywords(news):
    try:
        title = news.find('a', class_='post__title_link').get_text()
        anounce = news.find('p').get_text()
        tags = news.find_all('a', class_='inline-list__item-link hub-link')
        
    except:
        return False
    
    tag_list = list(map(lambda x: x.get_text(), tags))
    tag_string = " ".join(tag_list).lower()
    for keyword in KEYWORDS:
        if keyword in title or keyword in anounce or keyword in tag_string:
            return True
    
    return False

def get_site_pars(url_site):
    date = []
    title = []
    url = []
    result = {}
    response = requests.get('https://habr.com/ru/all/')
    content = BS(response.text, 'html.parser')
    news = content.find_all('li', class_='content-list__item content-list__item_post shortcuts_item')
    for item in news:
        if check_keywords(item):
            try:
                date.append(item.find('span', class_='post__time').get_text())
                title.append(item.find('a', class_='post__title_link').get_text())
                url.append(item.find('a', class_='post__title_link').get('href'))
            except:
                pass


    result['date'] = date
    result['title'] = title
    result['url'] = url
    result_df = pd.DataFrame(result)
    
    return result_df

get_site_pars('https://habr.com/ru/all/')

,date,title,url
0,сегодня в 11:30,Датасет о мобильных приложениях,https://habr.com/ru/post/559094/


#### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.\
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.\
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>


In [121]:
"""
Добавил третью функцию:
- Получение контента все статьи
"""

KEYWORDS = ['python', 'парсинг']

def check_keywords(news):
    try:
        title = news.find('a', class_='post__title_link').get_text()
        anounce = news.find('p').get_text()
        tags = news.find_all('a', class_='inline-list__item-link hub-link')
        
    except:
        return False
    
    tag_list = list(map(lambda x: x.get_text(), tags))
    tag_string = " ".join(tag_list).lower()
    for keyword in KEYWORDS:
        if keyword in title or keyword in anounce or keyword in tag_string:
            return True
    
    return False

def get_articles(url):
    response = requests.get(url)
    article = BS(response.text, 'html.parser')
    text_article = article.find('div', class_='post__body post__body_full').get_text()

    return text_article.strip()

def get_site_pars(url_site):
    date = []
    title = []
    url = []
    article_text = []
    result = {}
    response = requests.get('https://habr.com/ru/all/')
    content = BS(response.text, 'html.parser')
    news = content.find_all('li', class_='content-list__item content-list__item_post shortcuts_item')
    for item in news:
        if check_keywords(item):
            try:
                date.append(item.find('span', class_='post__time').get_text())
                title.append(item.find('a', class_='post__title_link').get_text())
                url.append(item.find('a', class_='post__title_link').get('href'))
                article_text.append(get_articles(item.find('a', class_='post__title_link').get('href')))
            except:
                pass


    result['date'] = date
    result['title'] = title
    result['url'] = url
    result['text'] = article_text
    result_df = pd.DataFrame(result)
    
    return result_df

get_site_pars('https://habr.com/ru/all/')

,date,title,url,text
0,сегодня в 12:25,Как принимать платежи в Telegram | API Yoomone...,https://habr.com/ru/post/558924/,В данном посте мы научимся принимать платежи в...
1,сегодня в 11:48,Как мы запустили документооборот в Telegram и ...,https://habr.com/ru/post/558918/,Разбираем аргументы «за» и «против». В конце т...
2,сегодня в 10:00,В поисках упорядоченного множества в Python: р...,https://habr.com/ru/company/macloud/blog/558724/,"Множество (Set) — структура данных, которая по..."



#### Задание 2.
Обязательная часть\
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Check](https://www.avast.com/hackcheck/). Список email-ов задаем переменной в начале кода:\
\
EMAIL = [xxx@x.ru, yyy@y.com]\
\
В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.


In [215]:
"""
Сервис постоянно блокирует, пришлось сделать:
1) Добавить кучу заголовков
2) Сделать проверку достукивания до сервис с задержкой при проверке урла
3) Сделал задержку межде проверками
4) Между проверками сервис блочит, но из-за того что запросы идут с задержкой пока не достучится - то инфу получает
"""

EMAIL = ['xxx@x.ru', 'yyy@y.com']

for email in EMAIL:
    date = []
    source = []
    description = []
    result = {}
    
    i = 0
    url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    headers = {'Content-Type': 'application/json;charset=utf-8', 
               'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:88.0) Gecko/20100101 Firefox/88.0',
               'Vaar-Header-App-Build-Version': '1.0.0',
               'Vaar-Header-App-Product': 'hackcheck-web-avast',
               'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
               'Vaar-Version': '0'}
    email_request = {"emailAddresses":[email]}
    while True:
        response = requests.post(url, json=email_request, headers=headers)
        if response.text != 'null':
            if len(response.json()['breaches']) != 0:
                for item in response.json()['breaches']:
                    date.append(response.json()['breaches'][item]['publishDate'])
                    source.append(response.json()['breaches'][item]['site'])
                    description.append(response.json()['breaches'][item]['description'])
                
                result['date'] = date
                result['source'] = source
                result['description'] = description
                result_df = pd.DataFrame(result)
                print("email адрес", email ,"просочился")
                print(result_df)
            else:
                print("email адрес", email ,"чист")
            
            break
        
        else:
            print("Запрос заблокирован")
            time.sleep(2**i)
            i+=1
    time.sleep(5)

email адрес xxx@x.ru просочился
                   date            source  \
0  2019-03-28T00:00:00Z  verifications.io   
1  2020-05-21T00:00:00Z            vk.com   
2  2017-02-14T00:00:00Z    parapa.mail.ru   
3  2016-10-29T00:00:00Z            vk.com   
4  2016-10-21T00:00:00Z         adobe.com   
5  2017-02-14T00:00:00Z     cfire.mail.ru   
6  2017-01-31T00:00:00Z  cdprojektred.com   
7  2016-10-23T00:00:00Z         imesh.com   

                                         description  
0  Big data e-mail verification platform verifica...  
1  At some time in 2020, the Russian social netwo...  
2  In July and August 2016, two criminals execute...  
3  Popular Russian social networking platform VKo...  
4  In October of 2013, criminals penetrated Adobe...  
5  In July and August of 2016, two criminals carr...  
6  In March 2016, CDProjektRed.com.com's forum da...  
7  In June 2016, a cache of over 51 million user ...  
Запрос заблокирован
Запрос заблокирован
Запрос заблокирован
Запрос 

#### Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.\
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get\
\
GROUP = 'netology'  \
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ\
\
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [218]:
"""
Очень дурацкая документация по api у vk...
Но разобрался. Парсил посты из группы netology 
(ее идентификатор -30159897 - пришлось тоже покапаться как его найти)
Токен убрал
"""

params = {'owner_id': '-30159897',
          'v': '5.131',
          'access_token': '********************'}

url = 'https://api.vk.com/method/wall.get'

response = requests.get(url, params=params)
dates = []
posts = []
result = {}
for item in response.json()['response']['items']:
    dates.append(dt.fromtimestamp(item['date']))
    posts.append(item['text'])

result['date'] = dates
result['post'] = posts

result_df = pd.DataFrame(result)
result_df

,date,post
0,2021-05-21 10:17:00,"Не знать, что делать на старте карьеры — норма..."
1,2021-05-25 11:33:00,Соцсети — не только место для общения с друзья...
2,2021-05-24 17:12:00,События 2020 года кардинально изменили экономи...
3,2021-05-24 11:01:00,📚 31 мая стартует курс «Python-разработка для ...
4,2021-05-23 12:05:00,Над созданием любой видеоигры работают програм...
5,2021-05-22 18:31:00,"В афише — события для дизайнеров, программисто..."
6,2021-05-22 11:26:00,Как бы Инстаграм и ТикТок ни пытались захватит...
7,2021-05-21 16:00:09,Вместе с коллегами из vc.ru запускаем серию оп...
8,2021-05-20 18:49:00,"Причин, по которым мы в три часа ночи скроллим..."
9,2021-05-20 11:14:00,В очередном выпуске НетоNews поговорим о путеш...
